In [2]:
import numpy as np
import random
from datetime import datetime, timedelta, timezone
from sgp4.api import Satrec
from sgp4.conveniences import jday

# Constants
MU = 398600.4418  # Earth's gravitational parameter, km^3/s^2

# ----------------------------
# Step 1: Read Random TLE
# ----------------------------
def read_random_tle(filename):
    with open(filename, "r") as file:
        lines = file.readlines()

    tle_sets = []
    for i in range(0, len(lines) - 2, 3):
        name = lines[i].strip()
        line1 = lines[i + 1].strip()
        line2 = lines[i + 2].strip()
        if line1.startswith("1 ") and line2.startswith("2 "):
            tle_sets.append((name, line1, line2))

    if not tle_sets:
        raise ValueError("No valid TLEs found in the file.")

    return random.choice(tle_sets)

# ----------------------------
# Step 2: Gravity Acceleration
# ----------------------------
def gravitational_acceleration(r):
    norm_r = np.linalg.norm(r)
    return -MU * r / norm_r**3

# ----------------------------
# Step 3: RK4 Integrator
# ----------------------------
def rk4_step(r, v, dt):
    k1_v = gravitational_acceleration(r)
    k1_r = v

    k2_v = gravitational_acceleration(r + 0.5 * dt * k1_r)
    k2_r = v + 0.5 * dt * k1_v

    k3_v = gravitational_acceleration(r + 0.5 * dt * k2_r)
    k3_r = v + 0.5 * dt * k2_v

    k4_v = gravitational_acceleration(r + dt * k3_r)
    k4_r = v + dt * k3_v

    r_next = r + dt / 6 * (k1_r + 2*k2_r + 2*k3_r + k4_r)
    v_next = v + dt / 6 * (k1_v + 2*k2_v + 2*k3_v + k4_v)

    return r_next, v_next

# ----------------------------
# Step 4: Simulate Orbit
# ----------------------------
def simulate_orbit_rk4(tle1, tle2, dt=10.0, max_minutes=180):
    satellite = Satrec.twoline2rv(tle1, tle2)
    now = datetime.now(timezone.utc)

    jd, fr = jday(now.year, now.month, now.day, now.hour, now.minute, now.second + now.microsecond / 1e6)
    e, r0, v0 = satellite.sgp4(jd, fr)

    if e != 0:
        raise ValueError(f"SGP4 error code {e}")

    r = np.array(r0)
    v = np.array(v0)
    r_start = r.copy()

    orbit = [(now.isoformat(), *r)]

    for _ in range(int((max_minutes * 60) / dt)):
        r, v = rk4_step(r, v, dt)
        now += timedelta(seconds=dt)
        orbit.append((now.isoformat(), *r))

        if np.linalg.norm(r - r_start) < 20 and len(orbit) > 10:
            break

    return orbit

# ----------------------------
# Step 5: Save to Structured .txt
# ----------------------------
def save_positions_as_structured_output(positions, name="CALSPHERE_SIM", output_filename="RK4_orbit.txt"):
    with open(output_filename, "w") as f:
        f.write(f"{name}\n")
        for t, x, y, z in positions:
            f.write(f"{t:<26} {x:>10.3f} {y:>10.3f} {z:>10.3f}\n")

# ----------------------------
# Step 6: Run All Together
# ----------------------------
if __name__ == "__main__":
    tle_file = "gp.txt"  # replace this with your actual file
    name, line1, line2 = read_random_tle(tle_file)
    print(f"Propagating satellite: {name}")

    orbit_data = simulate_orbit_rk4(line1, line2)
    save_positions_as_structured_output(orbit_data, name=name)

    print(f"Saved {len(orbit_data)} propagated points to RK4_orbit.txt")


Propagating satellite: JILIN-1 GAOFEN 03D09
Saved 1081 propagated points to RK4_orbit.txt
